In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.builder.build_dataflow_config import ShellFlowType
import os
import shutil
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
import finn.transformation.streamline.absorb as absorb
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.general import GiveUniqueNodeNames
from finn.core.modelwrapper import ModelWrapper
from finn.builder.build_dataflow_config import DataflowBuildConfig
from finn.transformation.streamline import Streamline

model_file = "classifier_1.onnx"

rtlsim_output_dir = "u280_classifier_5"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")
    
def step_pre_streamline(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(Change3DTo4DTensors())
    model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
    model = model.transform(Streamline())
    return model


def step_convert_final_layers(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(to_hls.InferChannelwiseLinearLayer())
    model = model.transform(to_hls.InferLabelSelectLayer())
    model = model.transform(GiveUniqueNodeNames())
    return model
    
def select_build_steps():
    return [
        "step_qonnx_to_finn",
        "step_tidy_up",
        step_pre_streamline,
        "step_streamline",
        "step_convert_to_hls",
        step_convert_final_layers,
        "step_create_dataflow_partition",
        "step_target_fps_parallelization",
        "step_apply_folding_config",
        "step_generate_estimate_reports",
        "step_hls_codegen",
        "step_hls_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        #"step_measure_rtlsim_performance",
        #"step_out_of_context_synthesis",
        #"step_synthesize_bitfile",
        #"step_make_pynq_driver",
        #"step_deployment_package",
    ]

In [2]:
cfg_stitched_ip = build.DataflowBuildConfig(
    steps               = select_build_steps(),
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    
    #board = "U250",
    fpga_part           = "xcu280-fsvh2892-2L-e",
    vitis_platform      = "xilinx_u280_xdma_201920_3",
    shell_flow_type     = ShellFlowType.VITIS_ALVEO,    
    #fpga_part           = "xc7z020clg400-1",
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.OOC_SYNTH,
        #build_cfg.DataflowOutputType.BITFILE,
    ]
    
)
"""
    verify_steps = [
        build_cfg.VerificationStepType.QONNX_TO_FINN_PYTHON,
        build_cfg.VerificationStepType.TIDY_UP_PYTHON,
        build_cfg.VerificationStepType.STREAMLINED_PYTHON,
        build_cfg.VerificationStepType.FOLDED_HLS_CPPSIM,
        build_cfg.VerificationStepType.STITCHED_IP_RTLSIM,
    ],
    verify_input_npy = "in_single.npy",
    verify_expected_output_npy = "out_single.npy"
    """

'\n    verify_steps = [\n        build_cfg.VerificationStepType.QONNX_TO_FINN_PYTHON,\n        build_cfg.VerificationStepType.TIDY_UP_PYTHON,\n        build_cfg.VerificationStepType.STREAMLINED_PYTHON,\n        build_cfg.VerificationStepType.FOLDED_HLS_CPPSIM,\n        build_cfg.VerificationStepType.STITCHED_IP_RTLSIM,\n    ],\n    verify_input_npy = "in_single.npy",\n    verify_expected_output_npy = "out_single.npy"\n    '

In [3]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

Building dataflow accelerator from classifier_1.onnx
Intermediate outputs will be generated in /tmp/finn_dev_neni
Final outputs will be generated in u280_classifier_5
Build log is at u280_classifier_5/build_dataflow.log
Running step: step_qonnx_to_finn [1/14]
Running step: step_tidy_up [2/14]
Running step: step_pre_streamline [3/14]
Running step: step_streamline [4/14]
Running step: step_convert_to_hls [5/14]
Running step: step_convert_final_layers [6/14]
Running step: step_create_dataflow_partition [7/14]
Running step: step_target_fps_parallelization [8/14]
Running step: step_apply_folding_config [9/14]
Running step: step_generate_estimate_reports [10/14]
Running step: step_hls_codegen [11/14]
Running step: step_hls_ipgen [12/14]
Running step: step_set_fifo_depths [13/14]
Running step: step_create_stitched_ip [14/14]
Completed successfully
CPU times: user 12.9 s, sys: 0 ns, total: 12.9 s
Wall time: 1h 15min 54s


0

In [14]:
from finn.util.visualization import showInNetron
showInNetron(rtlsim_output_dir + "/intermediate_models/step_convert_final_layers.onnx")

FileNotFoundError: [Errno 2] No such file or directory: 'u250_classifier/intermediate_models/step_convert_final_layers.onnx'